In [32]:
from starlette.testclient import TestClient
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass, field
from typing import List, Literal

In [33]:
from dataclasses import dataclass, field
from typing import List
from enum import Enum

MIN_GAMES = 10
MIN_APPS = 10

class AppsType(Enum):
    APPS = 'apps'
    STARTS = 'starts'

@dataclass
class Season:
    season: int

    def __post_init__(self):
        if not 1921 <= self.season <= 2024:
            raise ValueError('Invalid season year')

@dataclass
class LeagueTiers:
    league_tiers: List[int] = field(default_factory=list)

@dataclass
class CupCompetitions:
    cup_competitions: List[str] = field(default_factory=list)

@dataclass
class Venues:
    venues: List[str] = field(default_factory=list)

@dataclass(kw_only=True)
class AllRecords:
    min_season: Season
    max_season: Season
    league_tiers: LeagueTiers
    cup_competitions: CupCompetitions
    venues: Venues
    inc_playoffs: bool = field(default=True)
    pens_as_draw: bool = field(default=True)

@dataclass
class SeasonRecordsAll(AllRecords):    
    min_game_no: int = field(default=1)
    max_game_no: int = field(default=65)

@dataclass
class Head2HeadRecords(AllRecords):
    team_name: str
    min_meetings: int = field(default=MIN_GAMES)

@dataclass
class Head2HeadRecordsAll(AllRecords):
    min_meetings: int = field(default=MIN_GAMES)

@dataclass
class ManagerRecords(AllRecords):
    manager_name: str

@dataclass
class ManagerRecordsAll(AllRecords):
    min_games: int = field(default=MIN_GAMES)

@dataclass
class PlayerRecords(AllRecords):
    player_name: str

@dataclass
class PlayerRecordsAll(AllRecords):
    min_apps: int = field(default=MIN_APPS)
    apps_or_starts: str = field(default='starts')

In [19]:
app, rt = fast_app(pico=True)

server = JupyUvi(app)

In [20]:
radio = Fieldset(
    Legend('Include play-off games?'),
    Input(type='radio', name='inc_playoffs', value=1, checked=True),
    Label('Yes'),
    Input(type='radio', name='inc_playoffs', value=0),
    Label('No')
)

show(radio)

In [21]:
ssns = [1921, 2024]

range = Input(
    name=f"min_ssn",
    type="range",
    value=f"1921",
    placeholder=f"Min season",
    min=1921,
    max=2021,
    step=1,
    id=f"min_ssn"
)

input_range = (Div(
    '1921', 
    Input(
        type="range",
        name=f"min_ssn",
        value=f"1921",
        placeholder=f"1921",
        min=1921,
        max=2024,
        step=1,
        id=f"min_ssn",
        hx_on_input=f"document.getElementById('min_ssn_display').innerText = this.value;"
    ),
    Span(min(ssns), id="min_ssn_display")
),
Div(
    Input(
        type="range",
        name=f"max_ssn",
        value=f"2024",
        placeholder=f"Max.",
        min=1921,
        max=2024,
        step=1,
        id=f"id",
        hx_on_input=f"document.getElementById('max_ssn_display').innerText = this.value;"
    ),
    Span(max(ssns), id="max_ssn_display")
))

show(input_range)

In [22]:
show(
    Input(
        type="range",
        name=f"min_ssn",
        value=f"1921",
        placeholder=f"1921",
        min=1921,
        max=2024,
        step=1,
        id=f"min_ssn",
        hx_on_input=f"document.getElementById('min_ssn_display').innerText = this.value;"
    ),
    Span(f"1921", id=f"min_ssn_display")
)

In [23]:
input_nos = Div(
    Input(
        name=f"min_season",
        type="number",
        value=min(ssns),
        placeholder=f"Minimum season",
        min=min(ssns),
        max=max(ssns),
        step=1,
        id=f"min_season"
    )," - ",
    Input(
        name=f"max_season",
        type="number",
        value=max(ssns),
        placeholder=f"Maximum season",
        min=min(ssns),
        max=max(ssns),
        step=1,
        id=f"max_season"
    )
)

show(input_nos)

In [24]:
league_tiers = {
    2: '2: Championship',
    3: '3: League One',
    4: '4: League Two',
    5: '5: National League'
}

def checkboxes(field: str, items: dict):
        checkboxes = [(Label(CheckboxX(
            name=field,
            value=k,
            checked=True
        ), v)) for k, v in items.items()]
    
        return Div(*checkboxes)

lge_checkboxes = checkboxes('league_tiers', league_tiers)

show(lge_checkboxes)

In [25]:
comps = {
    'Anglo-Italian Cup': 'Anglo-Italian Cup',
    'Associate Members\' Cup': 'Associate Members\' Cup',
    'FA Cup': 'FA Cup',
    'FA Trophy': 'FA Trophy',
    'Full Members\' Cup': 'Full Members\' Cup',
    'League Cup': 'League Cup',
    'War League': 'War League'
}

def checkboxes(field: str, items: dict):
        checkboxes = [(Label(CheckboxX(
            name=field,
            value=k,
            checked=True
        ), v)) for k, v in items.items()]
    
        return Div(*checkboxes)

comp_checkboxes = checkboxes('comps', comps)

show(comp_checkboxes)

In [26]:
venues = {
    'H': 'Home',
    'A': 'Away',
    'N': 'Neutral'
}

def checkboxes(field: str, items: dict):
        checkboxes = [(Label(CheckboxX(
            name=field,
            value=k,
            checked=True
        ), v)) for k, v in items.items()]
    
        return Div(*checkboxes)

venue_checkboxes = checkboxes('venues', venues)

show(venue_checkboxes)

In [27]:
@rt("/")
def get():
    return Form(
        input_nos,
        input_range,
        Hr(),
        lge_checkboxes,
        radio,
        Hr(),
        comp_checkboxes,
        Hr(),
        venue_checkboxes,
    )

In [28]:
@rt("/radios")
async def post(request):
    return Div(
        f'{await request.form()}'
    )

In [29]:
HTMX()